In [32]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

%matplotlib inline

from tqdm import tqdm

In [33]:
# !unzip mmml-data.zip

In [34]:
ratings = pd.read_csv('mmml-data/ratings.csv')
movies = pd.read_csv('mmml-data/movies.csv')

In [35]:
df_merged = pd.merge(ratings,movies,on='movieId')
df_merged = df_merged.drop(["genres"], axis=1)
df_merged.head()

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story (1995)
1,5,1,4.0,847434962,Toy Story (1995)
2,7,1,4.5,1106635946,Toy Story (1995)
3,15,1,2.5,1510577970,Toy Story (1995)
4,17,1,4.5,1305696483,Toy Story (1995)


In [36]:
df_merged.groupby('title')['rating'].count().sort_values(ascending=False).head()

title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: rating, dtype: int64

In [37]:
ratings_separate = pd.DataFrame(df_merged.groupby('title')['rating'].mean())
ratings_separate.head()

,rating
title,
'71 (2014),4.0
'Hellboy': The Seeds of Creation (2004),4.0
'Round Midnight (1986),3.5
'Salem's Lot (2004),5.0
'Til There Was You (1997),4.0


In [38]:
ratings_separate['num_ratings'] = pd.DataFrame(df_merged.groupby('title')['rating'].count())
ratings_separate.head()

,rating,num_ratings
title,,
'71 (2014),4.0,1
'Hellboy': The Seeds of Creation (2004),4.0,1
'Round Midnight (1986),3.5,2
'Salem's Lot (2004),5.0,1
'Til There Was You (1997),4.0,2


### NMF (from scratch)

Here we are preparing the matrix for further decomposition, and filling all the missing entries with zeros.

In [39]:
ratings_nmf = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
ratings_nmf.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The next step is implemening the function to decompose matrix of user-movie ratings into two matrices using Non-Negative Matrix Factorization.

In [40]:
def nmf_recommendation(R, n_latent_features, iterations=100, learning_rate=0.01, regularization=0.01):
    """
    Returns:
    P - user-feature matrix
    Q - feature-item matrix
    """

    num_users, num_items = R.shape

    P = np.random.rand(num_users, n_latent_features)
    Q = np.random.rand(n_latent_features, num_items)

    for iteration in tqdm(range(iterations), desc='Iterations:'):
        for i in range(num_users):
            for j in range(num_items):
                if R[i, j] > 0:
                    e_i_j = R[i, j] - np.dot(P[i, :], Q[:, j])
                    for k in range(n_latent_features):
                        P[i, k] = P[i, k] + learning_rate * (2 * e_i_j * Q[k, j] - regularization * P[i, k])
                        Q[k, j] = Q[k, j] + learning_rate * (2 * e_i_j * P[i, k] - regularization * Q[k, j])

        error = 0
        for i in range(num_users):
            for j in range(num_items):
                if R[i, j] > 0:
                    error += pow(R[i, j] - np.dot(P[i, :], Q[:, j]), 2)
                    for k in range(n_latent_features):
                        error += (regularization/2) * (pow(P[i, k], 2) + pow(Q[k, j], 2))

        if error < 0.001:
            break

    return P, Q

nmf_reccomendation() function takes matrix of user-movie ratings, number of latent features and other hyperparameters such as number of iterations, learning rate and regularization.

This algorithm performs iterative updates of the result matrices P and Q to try and minimize the error between actual and predicted (as product of P and Q) ratings.

In [41]:
P, Q = nmf_recommendation(ratings_nmf.values, 20, 100, 0.01, 0.01)

Iterations:: 100%|██████████| 100/100 [29:22<00:00, 17.63s/it]


In [42]:
predicted_ratings = np.dot(P, Q)

In [43]:
predicted_ratings

array([[4.30360277, 4.61767972, 4.69586948, ..., 3.49244172, 4.30583077,
        5.03924721],
       [3.09199027, 2.8829687 , 1.52549696, ..., 1.69781092, 3.05609296,
        2.98067598],
       [0.55382468, 2.68913396, 1.92525848, ..., 0.70701112, 2.89232842,
        3.06262591],
       ...,
       [2.57849943, 2.73091645, 2.27496697, ..., 2.78729906, 4.19573998,
        3.40053443],
       [3.40941679, 3.43498265, 2.92429162, ..., 3.39790544, 3.15167597,
        4.29165747],
       [4.39092119, 3.17909331, 3.19136093, ..., 2.76521802, 3.6398952 ,
        3.78282274]])

In [44]:
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=ratings_nmf.index, columns=ratings_nmf.columns)
predicted_ratings_df

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.303603,4.617680,4.695869,3.300181,4.181591,4.800936,5.261742,3.164115,2.115156,3.735665,...,3.861222,2.547067,5.191520,4.180687,3.906051,3.565882,3.570764,3.492442,4.305831,5.039247
2,3.091990,2.882969,1.525497,2.855507,1.422589,5.095306,4.050562,2.342572,2.089991,2.960788,...,3.191311,2.933157,2.567141,3.728577,2.522538,2.249468,2.067374,1.697811,3.056093,2.980676
3,0.553825,2.689134,1.925258,2.941671,0.592444,4.147724,4.471672,2.682836,1.593061,2.014219,...,1.003653,4.736328,2.587474,4.800060,1.849315,3.191440,2.839610,0.707011,2.892328,3.062626
4,5.826310,3.309464,5.489011,2.034204,4.068750,3.111501,1.663765,1.362745,3.035080,0.769779,...,3.142488,1.002787,3.586320,1.204326,3.516477,2.028026,1.225749,2.913085,1.330128,3.995831
5,3.902548,3.959023,2.684545,1.378441,1.012680,4.606525,2.450385,4.249737,2.097975,4.361862,...,4.796164,3.021336,3.909894,4.086036,3.071006,3.651380,2.742840,4.224949,3.240699,3.892205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,3.493711,3.814016,3.462113,2.797006,2.492943,4.227456,2.968338,2.537723,2.427367,3.085130,...,3.344331,2.412367,3.359279,3.770361,2.872258,3.206015,2.549467,3.339819,3.803391,4.052922
607,3.694442,3.654483,3.726738,2.013077,3.073751,3.975964,0.875824,3.362543,3.234072,4.203326,...,3.060192,1.746992,2.335651,3.234131,1.723103,2.385377,3.320879,2.874692,3.837634,3.561218
608,2.578499,2.730916,2.274967,2.701009,1.572791,4.325497,2.976493,2.752155,2.994915,3.782104,...,3.199391,3.102496,2.787616,3.946447,1.438963,2.557161,3.365719,2.787299,4.195740,3.400534


### NMF library

Here we are implementing NMF trough sklearn library.

In [45]:
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=100) 

nmf_model.fit(ratings_nmf.values)
P = nmf_model.transform(ratings_nmf.values)
Q = nmf_model.components_.T

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [46]:
R_pred = P.dot(Q.T)

In [47]:
R_pred

array([[4.15545791e+00, 0.00000000e+00, 3.70358252e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.44911733e-01, 0.00000000e+00, 0.00000000e+00, ...,
        2.36857276e-03, 2.36857276e-03, 5.57886462e-02],
       [1.08759708e-01, 7.42974465e-02, 6.02428529e-02, ...,
        0.00000000e+00, 0.00000000e+00, 4.13245292e-05],
       ...,
       [1.86171796e+00, 2.27623520e+00, 1.82129951e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.04525662e-06],
       [7.84872760e-01, 5.71270089e-01, 3.23799869e-02, ...,
        6.22762795e-04, 6.22762795e-04, 0.00000000e+00],
       [5.05498012e+00, 5.18515950e-03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.04567807e-04]])

In [48]:
predicred_library_df = pd.DataFrame(R_pred, index=ratings_nmf.index, columns=ratings_nmf.columns)
predicred_library_df

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.155458,0.000000,3.703583,0.000000,0.011773,4.252076,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.244912,0.000000,0.000000,0.000000,0.051946,0.099310,0.005369,0.000000,0.000057,0.100506,...,0.002369,0.002030,0.002707,0.002707,0.002369,0.002707,0.002369,0.002369,0.002369,0.055789
3,0.108760,0.074297,0.060243,0.000000,0.000236,0.083454,0.021039,0.000661,0.000804,0.074512,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000041
4,1.697014,0.430353,0.157525,0.091270,0.069792,1.086067,0.138083,0.048071,0.000000,0.623204,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.873699,0.815525,0.129626,0.182477,0.234465,0.572866,0.301970,0.127410,0.000007,1.467921,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.469339,0.001428,0.000000,0.000000,0.000000,0.000000,2.444371,0.000000,0.000000,0.019533,...,0.000142,0.000122,0.000162,0.000162,0.000142,0.000162,0.000142,0.000142,0.000142,0.000000
607,2.244610,0.926438,0.571617,0.019151,0.164699,1.250316,0.178625,0.014938,0.210636,1.412235,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
608,1.861718,2.276235,1.821300,0.000000,0.000000,0.232703,0.000494,0.000000,0.112805,3.899578,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000001


### Evaluation metric (RMSE - Root Mean Squared Error)

We will evaluate the algorithms through RMSE (Root Mean Squared Error) metric

In [49]:
def compute_rmse(actual_ratings, predicted_ratings):
    actual = actual_ratings.flatten()
    predicted = predicted_ratings.flatten()

    squared_error = np.power(actual - predicted, 2)
    mse = np.mean(squared_error)

    rmse = np.sqrt(mse)
    return rmse

For our implementation:

In [50]:
rmse_predicted = compute_rmse(ratings_nmf.values, predicted_ratings)
rmse_predicted

3.5339460531188367

For implementation from library (sklearn)

In [51]:
rmse_sklearn = compute_rmse(ratings_nmf.values, R_pred)
rmse_sklearn

0.2684874632522332

## ALS

Here we have our implementation of Alternating Least Squares.

The main idea of the algorithm is to minimise the error, while performing two loops, one for updating X, while fixing Y and the other for updating Y, while fixing X.

In [109]:
def als_recommendation(R, k=10, epochs=10, _lambda=0.1, tol=1e-4):
    # Initialize X and Y first, might be random, we will use a matrix of 1's here
    X = np.ones((R.shape[0], k))
    Y = np.ones((R.shape[1], k))
    
    for epoch in range(epochs):
        prev_X = np.copy(X)
        prev_Y = np.copy(Y)
        
        # Update X while fixing Y
        for i in range(R.shape[0]):
            YTY = np.matmul(Y.T, Y)
            YTR = np.matmul(Y.T, R[i, :].T)
            X[i, :] = np.linalg.solve(YTY + _lambda * np.eye(k), YTR)
        
        # Update Y while fixing X
        for j in range(R.shape[1]):
            XTX = np.matmul(X.T, X)
            XTR = np.matmul(X.T, R[:, j])
            Y[j, :] = np.linalg.solve(XTX + _lambda * np.eye(k), XTR)
        
        # Check convergence
        if np.linalg.norm(X - prev_X) < tol and np.linalg.norm(Y - prev_Y) < tol:
            break
        
    return X, Y

In [110]:
X, Y = als_recommendation(ratings_nmf.values)

In [111]:
predicted_ratings_als = np.dot(X, Y.T)

In [112]:
predicted_ratings_als_df = pd.DataFrame(predicted_ratings_als, index=ratings_nmf.index, columns=ratings_nmf.columns)
predicted_ratings_als_df

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,2.500782,0.958878,1.015994,-0.022805,0.204819,1.554388,0.140007,-0.013059,0.165463,2.022797,...,-0.008743,-0.007494,-0.009992,-0.009992,-0.008743,-0.009992,-0.008743,-0.008743,-0.008743,-0.021409
2,0.074577,0.012161,-0.016875,0.001103,0.018670,0.045523,-0.036267,0.008385,-0.003938,-0.051562,...,0.007580,0.006497,0.008663,0.008663,0.007580,0.008663,0.007580,0.007580,0.007580,0.012009
3,0.018826,0.021197,0.023378,-0.004352,-0.015268,0.071863,-0.010505,0.000901,0.009684,0.063974,...,0.000548,0.000470,0.000626,0.000626,0.000548,0.000626,0.000548,0.000548,0.000548,-0.001815
4,1.719815,0.128117,0.166722,0.048293,0.162720,0.493286,0.217413,-0.063280,-0.056040,0.044609,...,-0.008373,-0.007177,-0.009569,-0.009569,-0.008373,-0.009569,-0.008373,-0.008373,-0.008373,-0.008562
5,1.302968,0.929190,0.380172,0.123970,0.518012,0.799678,0.619095,0.112455,0.109634,1.142925,...,-0.000469,-0.000402,-0.000536,-0.000536,-0.000469,-0.000536,-0.000469,-0.000469,-0.000469,-0.000866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,1.015009,2.139176,0.088143,0.170515,0.510457,0.190454,1.584801,-0.093062,-0.032795,-0.291682,...,0.011541,0.009892,0.013190,0.013190,0.011541,0.013190,0.011541,0.011541,0.011541,0.011696
607,2.620620,0.939539,0.618334,0.031504,0.292603,1.577736,0.343427,0.026079,0.142496,1.811645,...,-0.005869,-0.005031,-0.006707,-0.006707,-0.005869,-0.006707,-0.005869,-0.005869,-0.005869,-0.014544
608,2.599534,1.985237,1.891266,-0.074848,0.496898,2.967910,0.526408,0.366087,0.232110,3.809046,...,-0.050531,-0.043312,-0.057750,-0.057750,-0.050531,-0.057750,-0.050531,-0.050531,-0.050531,-0.001070


### Evaluation with RMSE

In [113]:
rmse_als = compute_rmse(ratings_nmf.values, predicted_ratings_als)
rmse_als

0.3755815557402783